In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Demo to Show Anywidget mode

In [2]:
import bigframes.pandas as bpd

Set the display option to use anywidget

In [3]:
bpd.options.bigquery.ordering_mode = "partial"
bpd.options.display.repr_mode = "anywidget"

Load Sample Data

In [4]:
df = bpd.read_gbq("bigquery-public-data.usa_names.usa_1910_2013")
print(df)

Computation deferred. Computation will process 171.4 MB


Display Series in anywidget mode

In [5]:
test_series = df["year"]
print(test_series)

Computation deferred. Computation will process 44.4 MB


Display with Pagination

In [6]:
df

TableWidget(page_size=10, row_count=5552452, table_html='<table border="1" class="dataframe table table-stripe…

Computation deferred. Computation will process 171.4 MB

Programmatic Navigation Demo

In [7]:
from bigframes.display.anywidget import TableWidget
import math
  
# Create widget programmatically  
widget = TableWidget(df)
print(f"Total pages: {math.ceil(widget.row_count / widget.page_size)}")
  
# Display the widget
widget

Total pages: 555246


TableWidget(page_size=10, row_count=5552452, table_html='<table border="1" class="dataframe table table-stripe…

Test Navigation Programmatically

In [8]:
# Simulate button clicks programmatically
print("Current page:", widget.page)

# Go to next page
widget.page = 1
print("After next:", widget.page)

# Go to previous page
widget.page = 0
print("After prev:", widget.page)

Current page: 0
After next: 1
After prev: 0


Edge Case Demonstration

In [ ]:
# Test with very small dataset
small_df = df.sort_values(["name", "year", "state"]).head(5)
small_widget = TableWidget(small_df)
print(f"Small dataset pages: {math.ceil(small_widget.row_count / small_widget.page_size)}")
small_widget